In [1]:
import africanus
import dask

In [2]:
from daskms import xds_from_ms
dadasets = xds_from_ms('/fastpool/data/W-15int-8000chan/01.ms', chunks={'row': 2e3})

In [4]:
dadasets[0].DATA

<xarray.DataArray 'DATA' (row: 2096128, chan: 8000, corr: 4)>
dask.array<read~DATA~[0,0]-01.ms, shape=(2096128, 8000, 4), dtype=complex64, chunksize=(2000, 8000, 4), chunktype=numpy.ndarray>
Coordinates:
    ROWID    (row) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
Dimensions without coordinates: row, chan, corr

In [8]:
from casacore import tables

In [10]:
tables.table('/fastpool/data/W-15int-8000chan/00.ms/SPECTRAL_WINDOW', ack=False).getcol('CHAN_FREQ')

array([[7.0000000e+08, 7.0016250e+08, 7.0032500e+08, ..., 1.9995125e+09,
        1.9996750e+09, 1.9998375e+09]])

In [6]:
dadasets[0]

<xarray.Dataset>
Dimensions:         (row: 2096128, chan: 8000, corr: 4, uvw: 3)
Coordinates:
    ROWID           (row) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
Dimensions without coordinates: row, chan, corr, uvw
Data variables: (12/19)
    OBSERVATION_ID  (row) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
    INTERVAL        (row) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    FLAG_ROW        (row) bool dask.array<chunksize=(2000,), meta=np.ndarray>
    EXPOSURE        (row) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    FLAG            (row, chan, corr) bool dask.array<chunksize=(2000, 8000, 4), meta=np.ndarray>
    FEED1           (row) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
    ...              ...
    TIME_CENTROID   (row) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    DATA            (row, chan, corr) complex64 dask.array<chunksize=(2000, 8000, 4), meta=np.ndarray>
    TIME            (row) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    UVW             (row, uvw) float64 dask.array<chunksize=(2000, 3), meta=np.ndarray>
    ANTENNA2        (row) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
    WEIGHT          (row, corr) float32 dask.array<chunksize=(2000, 4), meta=np.ndarray>
Attributes:
    __daskms_partition_schema__:  (('FIELD_ID', 'int32'), ('DATA_DESC_ID', 'i...
    FIELD_ID:                     0
    DATA_DESC_ID:                 0

In [4]:
from daskms import xds_to_table

In [5]:
dadasets[0].DATA.data = dadasets[0].MODEL_DATA.data
write = xds_to_table(dadasets, '/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/01.ms', 'DATA')

In [6]:
%time dask.compute(write)

CPU times: user 34.2 s, sys: 2min 19s, total: 2min 53s
Wall time: 6min 37s


(<xarray.Dataset>
 Dimensions:  (chan: 1, corr: 1, row: 21)
 Dimensions without coordinates: chan, corr, row
 Data variables:
     DATA     (row, chan, corr) bool True True True True ... True True True True,)

In [5]:
import numpy as np

In [12]:
from casacore.tables import table

for i in range(15):
    with table(f'/lustre/yuping/2020-12-28-dsa2000-take2/130chan_60x15s_bright/{i:02d}.ms', ack=False) as t:
        weights = t.getcol('WEIGHT', nrow=4000)
        flags = t.getcol('FLAG', nrow=4000)
        print(i, np.unique(weights), np.unique(flags))

0 [1.] [False  True]
1 [1.] [False  True]
2 [1.] [False  True]
3 [1.] [False]
4 [1.] [False]
5 [1.] [False]
6 [1.] [False]
7 [1.] [False]
8 [1.] [False]
9 [1.] [False]
10 [1.] [False]
11 [1.] [False]
12 [1.] [False]
13 [1.] [False  True]
14 [1.] [False  True]


In [13]:
for i in range(15):
    with table(f'/lustre/yuping/2020-12-28-dsa2000-take2/130chan_60x15s_random/{i:02d}.ms', ack=False) as t:
        weights = t.getcol('WEIGHT', nrow=4000)
        flags = t.getcol('FLAG', nrow=4000)
        print(i, np.unique(weights), np.unique(flags))

0 [1.] [False  True]
1 [1.] [False  True]
2 [1.] [False  True]
3 [1.] [False]
4 [1.] [False]
5 [1.] [False]
6 [1.] [False]
7 [1.] [False]
8 [1.] [False]
9 [1.] [False]
10 [1.] [False]
11 [1.] [False]
12 [1.] [False]
13 [1.] [False  True]
14 [1.] [False  True]


In [14]:
for i in range(15):
    with table(f'/lustre/yuping/2020-12-28-dsa2000-take2/130chan_60x15s/{i:02d}.ms', ack=False) as t:
        weights = t.getcol('WEIGHT', nrow=4000)
        flags = t.getcol('FLAG', nrow=4000)
        print(i, np.unique(weights), np.unique(flags))

0 [1.] [False  True]
1 [1.] [False  True]
2 [1.] [False  True]
3 [1.] [False]
4 [1.] [False]
5 [1.] [False]
6 [1.] [False]
7 [1.] [False]
8 [1.] [False]
9 [1.] [False]
10 [1.] [False]
11 [1.] [False]
12 [1.] [False]
13 [1.] [False  True]
14 [1.] [False  True]


In [15]:
with table(f'/lustre/yuping/2020-12-28-dsa2000-take2/130chan_60x15s/00.ms', ack=False) as t:
    flags = t.getcol('FLAG', nrow=4000)

In [21]:
flags.shape

(4000, 130, 4)

In [23]:
s = np.sum(np.sum(flags,axis=-1), axis = -1)

In [24]:
s

array([  0,   0,   0, ..., 520, 520, 520])

In [25]:
s.shape

(4000,)

In [26]:
np.sum(s>0)

29

In [27]:
t = table(f'/lustre/yuping/2020-12-28-dsa2000-take2/130chan_60x15s_random/00.ms', ack=False)

In [29]:
dat = t.getcol('DATA', nrow=200)

In [30]:
corrected = t.getcol('CORRECTED_DATA', nrow=200)

In [32]:
dat - corrected

array([[[-5.8934093e-06-5.8964128e-05j,  0.0000000e+00+0.0000000e+00j,
          0.0000000e+00+0.0000000e+00j, -5.8934093e-06-5.8964128e-05j],
        [-5.9716403e-06-5.8895675e-05j,  0.0000000e+00+0.0000000e+00j,
          0.0000000e+00+0.0000000e+00j, -5.9716403e-06-5.8895675e-05j],
        [-6.0498714e-06-5.8826292e-05j,  0.0000000e+00+0.0000000e+00j,
          0.0000000e+00+0.0000000e+00j, -6.0498714e-06-5.8826292e-05j],
        ...,
        [-1.6257167e-05-4.2994507e-05j,  0.0000000e+00+0.0000000e+00j,
          0.0000000e+00+0.0000000e+00j, -1.6257167e-05-4.2994507e-05j],
        [-1.6327016e-05-4.2839907e-05j,  0.0000000e+00+0.0000000e+00j,
          0.0000000e+00+0.0000000e+00j, -1.6327016e-05-4.2839907e-05j],
        [-1.6396865e-05-4.2685308e-05j,  0.0000000e+00+0.0000000e+00j,
          0.0000000e+00+0.0000000e+00j, -1.6396865e-05-4.2685308e-05j]],

       [[ 4.0978193e-07+1.3568613e-05j,  0.0000000e+00+0.0000000e+00j,
          0.0000000e+00+0.0000000e+00j,  4.0978193e-07+1